## Francony Alexandre

# WebScrapper for MTG decks on cardmarket.com

Nous allons dans un premier temps nous assurer que le site est atteignable à l'aide de la librairie requests et de la fonction get_status_code. Cette fonction sera réutilisée dans le code pour vérifier que le site est toujours atteignable avant chaque étape de récupération majeure.

In [1]:
import requests
import csv
import os
import requests
from bs4 import BeautifulSoup
import re

def get_status_code(url):
    response = requests.get(url)
    return response.status_code

url = "https://www.cardmarket.com/fr/Magic/Products/Singles/Phyrexia-All-Will-Be-One"
status_code = get_status_code(url)
print("Le code d'état de Cardmarket est", status_code)


Le code d'état de Cardmarket est 200


In [2]:
def prep(card):
    #transformer tout les caractères non alphanumériques en tirets et tout en minuscule et les espaces en tirets
    return re.sub(r'\W+', '-', card.replace("'","")).lower()

In [3]:
with open('data/deck.txt', 'r') as file:
    decklist = file.read()

#Récupérer la première ligne du fichier qui contient 'MIN_QUALITY = "<qualité>"'
MIN_QUALITY = decklist.splitlines()[0].split(" = ")[1].strip('"')

# Chercher la ligne contenant la variable LANGUAGES
for line in decklist.split('\n'):
    if line.startswith('LANGUAGES = '):
        # Extraire le contenu de la variable LANGUAGES
        LANGUAGES = eval(line.split('=')[1].strip())

#Récupérer chaque autre ligne du fichier qui contient "quantité nom de la carte" et les stocker dans un dictionaire
deck = {}
for line in decklist.splitlines()[2:]:
    quantity, card = line.split(" ", 1)
    deck[card] = quantity

#et maintenant ajoutons l'url de chaque carte dans le dictionnaire
for card in deck:
    deck[card] = [deck[card], "https://www.cardmarket.com/en/Magic/Cards/" + prep(card)]
#print(deck)


In [4]:
qualities = []
def switch_case_qual(argument):
    switcher = {
        "MT": ["MT"],
        "NM": ["NM", "MT"],
        "EX": ["EX", "NM", "MT"],
        "GD": ["GD", "EX", "NM", "MT"],
        "LP": ["LP", "GD", "EX", "NM", "MT"],
        "PL": ["PL", "LP", "GD", "EX", "NM", "MT"],
        "PO": ["PO", "PL", "LP", "GD", "EX", "NM", "MT"],
    }
    return switcher.get(argument, ["MT"])

qualities = switch_case_qual(MIN_QUALITY)
#print(qualities)


In [5]:
def getCardName(card):
    #récupérer le nom de la carte qui est la clé du dictionnaire
    card_name = card
    if(card_name == ''):
        card_name = 'Undefined'
    return card_name;

def getCardPrices(card):
    cards_sales = []
    tab_prices = []
    
    # Récupération du contenu de la page
    url = deck[card][1]
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Recherche de la liste des cartes
    table = soup.find_all('div', {'class': 'row no-gutters article-row'})

    for row in table:
        cards_sales.append(row)

    #récupérer les 5 premiers prix de vente de la carte
    for card in cards_sales:
        card_price = card.find('span', {"class" : "font-weight-bold color-primary small text-right text-nowrap"}).text.strip()[:-2]
        tab_prices.append(card_price.replace(',', '.'))
        if len(tab_prices) == 5:
            break

    return tab_prices;

def getCardQuantity(card):
    # récupérer l'url de la carte
    url = deck[card][1]
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Récupération du cadre contenant les informations de la carte
    card_frame = soup.find('dl', {"class" : "labeled row no-gutters mx-auto"})

    # Récupération de la quantité de la carte
    frame_infos = card_frame.find_all('dd', {"class" : "col-6 col-xl-7"})[-6:]

    # récupérer seulement la quantité, qui est contenu dans le premier élément du tableau
    card_quantity = frame_infos[0].text.strip()

    return card_quantity;

def getCardAveragePrice(card):
    # Récupération du contenu de la page
    url = deck[card][1]
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Récupération du cadre contenant les informations de la carte
    card_frame = soup.find('dl', {"class" : "labeled row no-gutters mx-auto"})

    # Récupération des données en chiffre de la carte
    frame_infos = card_frame.find_all('dd', {"class" : "col-6 col-xl-7"})[-6:]
    
    # récupérer seulement le prix moyen sur 7 jours, qui est contenu dans l'avant dernier élément du tableau
    card_average_price = frame_infos[-2].text.strip()
    return card_average_price.replace(',', '.').replace(' €', '').replace('£', '');

def getBuyableInfo(card):
    cards_sales = {}

    # Récupération du contenu de la page
    url = deck[card][1]
    if(get_status_code(url) == 200):
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Recherche de la liste des cartes
        table = soup.find('div', {'class': 'table-body'})

        #dans table, trouver chaque ligne représentant une carte à vendre avec son prix, sa qualité et sa langue
        i=0
        for row in table:
            cards_sales[i] = [row.find('span', {"class" : "font-weight-bold color-primary small text-right text-nowrap"}).text.strip()[:-2], 
                            row.find_all('span', {"class" : "badge"})[1].text.strip(), 
                            row.find_all('span', {'class': 'icon mr-2'})[1].get('data-original-title')]
            i=i+1
    else:
        print("Le code d'état de la page de ", card, " est ", get_status_code(url), ", la page n'a pas pu être chargée.")
    return cards_sales;

def getCardImage(card):
    # Récupération du contenu de la page
    url = deck[card][1]
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Récupération de l'image de la carte
    card_image = soup.find('img', {"class" : "is-front"})

    # récupérer seulement l'url de l'image, qui est contenu dans l'attribut src de l'élément img
    card_image_url = "https:" + card_image.get('src')

    #Enregistrer l'image dans le dossier data/deckimages
    if not os.path.exists('data/deckimages'):
        os.makedirs('data/deckimages')

    img=requests.get(card_image_url)
    if "//" in card : 
        card = card.replace("//", "_")
    with open('data/deckimages/' + card + '.jpg', 'wb') as f:
        f.write(img.content)

    return card_image_url;

A présent que nous avons pu créer notre nouveau dictionnaire avec les cartes sélectionnées, nous allons pouvoir créer un nouveau fichier csv qui contiendra uniquement les infos des cartes sélectionnées et avoir plus rapidement les informations afin de pouvoir réagir presque en temps réel. Les cartes que j'ai sélectionnés sont purement arbitraire, je prévois de les inclure dans l'un de mes decks.

In [6]:
def getCardInfoWL(card):
    url = deck[card][1]
    if(get_status_code(url) == 200):
        card_intel = []
        card_intel.append(getCardName(card))
        card_intel.append(getCardPrices(card))
        card_intel.append(getCardQuantity(card))
        card_intel.append(getCardAveragePrice(card))
        card_intel.append(getBuyableInfo(card))

    else:
        print("Le code d'état de la page de ", card, " est ", get_status_code(url), ", la page n'a pas pu être chargée.")
    return card_intel

#for card in card_url:
#    print(getCardInfoWL(card))

In [8]:
total_price = 0
if not os.path.exists('data'):
    os.makedirs('data')

f = open('data/deckPrice.txt', 'w', encoding='utf-8')

for cards in deck:
    getCardImage(cards)
    this_card_quantity = int(deck[cards][0])
    #print("Carte : ", cards, " - Quantité : ", this_card_quantity)
    i = 0
    this_card_prices = getBuyableInfo(cards)
    #print(this_card_prices)
    while i < this_card_quantity:
        #parcourir le dictionnaire des prix de la carte
        for identifiant, intel in this_card_prices.items():
            #print("id:", identifiant, " - prix:", intel[0], " - qualité:", intel[1], " - langue:", intel[2])
            if i < this_card_quantity:
                #si la condition de la carte(2ème argument de l'élément price) est contenue dans MIN_QUALITY
                if intel[1] in qualities:
                    if intel[2] in LANGUAGES:
                        total_price += float(intel[0].replace(',', '.'))
                        i += 1
                        print("Carte ajoutée : ", cards, " - Prix : ", intel[0], " - Qualité : ", intel[1], " - Langue : ", intel[2], " - Quantité : ", i, "/", this_card_quantity)
                        f.write("Carte ajoutée : " + cards + " - Prix : " + intel[0] + " - Qualité : " + intel[1] + " - Langue : " + intel[2] + " - Quantité : " + str(i) + "/" + str(this_card_quantity)+"\n")
                        break
            else:
                break

to_write = "Prix total du deck : " + str(round(total_price, 2)) + "€"
f.write(to_write)

#afficher le prix total du deck avec 2 décimales
print("Prix total du deck : ", round(total_price, 2), "€")

Carte ajoutée :  Jodah, the Unifier  - Prix :  1,95  - Qualité :  NM  - Langue :  English  - Quantité :  1 / 1
Carte ajoutée :  Arcane Signet  - Prix :  0,30  - Qualité :  NM  - Langue :  French  - Quantité :  1 / 1
Carte ajoutée :  Arvad the Cursed  - Prix :  0,07  - Qualité :  NM  - Langue :  Spanish  - Quantité :  1 / 1
Carte ajoutée :  Atraxa, Grand Unifier  - Prix :  19,00  - Qualité :  NM  - Langue :  French  - Quantité :  1 / 1
Carte ajoutée :  Azusa, Lost but Seeking  - Prix :  3,00  - Qualité :  GD  - Langue :  French  - Quantité :  1 / 1
Carte ajoutée :  Bard Class  - Prix :  0,16  - Qualité :  NM  - Langue :  Spanish  - Quantité :  1 / 1
Carte ajoutée :  Beast Within  - Prix :  0,40  - Qualité :  NM  - Langue :  English  - Quantité :  1 / 1
Carte ajoutée :  Boros Charm  - Prix :  1,00  - Qualité :  EX  - Langue :  English  - Quantité :  1 / 1
Carte ajoutée :  Bountiful Promenade  - Prix :  3,79  - Qualité :  NM  - Langue :  English  - Quantité :  1 / 1
Carte ajoutée :  Capta